<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How Delete Unattached EBS Volume using unSkript legos.</b>
</div>

<br>

<center><h2>Delete Unattached EBS Volume</h2></center>

# Steps Overview
    1) Collecting the all EBS volumes which don't have any attachments.
    2) Delete the EBS volumes.

Here we will use unSkript Filter AWS Unattached EBS Volume Lego. This lego takes region as input. This inputs is used to filter and collect the EBS volumes which don't have any attachments.

In [3]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
import pprint

from beartype import beartype

@beartype
def aws_filter_ebs_unattached_volumes_printer(output):
    if output is None:
        return
    pprint.pprint({"Volume IDs": output})


@beartype
def aws_filter_ebs_unattached_volumes(handle, region: str) -> List:
    """aws_filter_ebs_unattached_volumes Returns an array of ebs volumes.

        :type region: string
        :param region: Used to filter the volume for specific region.

        :rtype: Result of the API in the List form.
    """
    # Filtering the volume by region
    ec2Client = handle.resource('ec2',region_name=region)
    volumes = ec2Client.volumes.all()

    # collecting the volumes which has zero attachments
    result=[]
    for volume in volumes:
        if len(volume.attachments) == 0:
            result.append(volume.id)

    return result


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "Region"
    }''')
task.configure(outputName="UnattachedVolumes")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_ebs_unattached_volumes, lego_printer=aws_filter_ebs_unattached_volumes_printer, hdl=hdl, args=args)

Here we will use unSkript Delete EBS Volume Lego. This lego takes region and volume_id as input. This inputs is used to delete the EBS volume.

In [11]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint


from beartype import beartype
@beartype
def aws_delete_volumes_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_delete_volumes(handle, volume_id: str, region: str) -> List:
    """aws_filter_ebs_unattached_volumes Returns an array of ebs volumes.

        :type region: string
        :param region: Used to filter the volume for specific region.

        :type volume_id: string
        :param volume_id: Volume ID needed to delete particular volume.

        :rtype: Result of the API in the List form.
    """
    result = []
    ec2Client = handle.client('ec2',region_name=region)

    # Adding logic for deletion criteria
    try:
        response = ec2Client.delete_volume(VolumeId=volume_id,)
        result.append(response)
    except Exception as e:
        result.append(e)

    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "Region",
    "volume_id": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "UnattachedVolumes",
    "iter_parameter": "volume_id"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(UnattachedVolumes) > 0",
    "condition_result": true
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_volumes, lego_printer=aws_delete_volumes_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS legos to filter unattached EBS volumes and delete those. To view the full platform capabilities of unSkript please visit https://unskript.com